## Reducers

Eles especificam como realizar atualizações.

Podemos usar o tipo $Annotated$ para especificar uma função redutora.

Por exemplo, neste caso, vamos adicionar o valor retornado de cada nó em vez de sobrescrevê-los.

Precisamos apenas de um redutor que possa realizar isso: `operator.add` é uma função do módulo `operator` do Python.

Quando `operator.add` é aplicado a listas, ele realiza a concatenação da lista.

In [ ]:
from operator import add
from typing import Annotated
from typing_extensions import TypedDict
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    foo: Annotated[list[int], add]

def node_1(state):
    print("---Node 1---")
    return {"foo": [state['foo'][-1] + 1]}

def node_2(state):
    print("---Node 2---")
    return {"foo": [state['foo'][-1] + 1]}

def node_3(state):
    print("---Node 3---")
    return {"foo": [state['foo'][-1] + 1]}

# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_1", "node_3")
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:

graph.invoke({"foo" : [1]})

## Custom Reducers

Também podemos definir redutores personalizados.

Por exemplo, vamos definir uma lógica de redutor personalizada para combinar listas e lidar com casos em que uma ou ambas as entradas podem ser nulas.

In [ ]:
def reduce_list(left: list | None, right: list | None) -> list:
    """Safely combine two lists, handling cases where either or both inputs might be None.

    Args:
        left (list | None): The first list to combine, or None.
        right (list | None): The second list to combine, or None.

    Returns:
        list: A new list containing all elements from both input lists.
               If an input is None, it's treated as an empty list.
    """
    if not left:
        left = []
    if not right:
        right = []
    return left + right

class DefaultState(TypedDict):
    foo: Annotated[list[int], add]

class CustomReducerState(TypedDict):
    foo: Annotated[list[int], reduce_list]

In [ ]:
def node_1(state):
    print("---Node 1---")
    return {"foo": [2]}

# Build graph
builder = StateGraph(DefaultState)
builder.add_node("node_1", node_1)

# Logic
builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

try:
    print(graph.invoke({"foo" : None}))
except TypeError as e:
    print(f"TypeError occurred: {e}")

Com custom reducer

In [ ]:
# Build graph
builder = StateGraph(CustomReducerState)
builder.add_node("node_1", node_1)

# Logic
builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

try:
    print(graph.invoke({"foo" : None}))
except TypeError as e:
    print(f"TypeError occurred: {e}")

## Messages

In [ ]:
from typing import Annotated
from langgraph.graph import MessagesState
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

# Defina um TypedDict personalizado que inclua uma lista de mensagens com o reducer add_messages.
class CustomMessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    added_key_1: str
    added_key_2: str
    # etc

# Use MessagesState, que inclui a chave messages, com o reducer add_messages.
class ExtendedMessagesState(MessagesState):
    # Adicione quaisquer chaves necessárias além das mensagens, que já estão pré-configuradas.
    added_key_1: str
    added_key_2: str
    # etc

In [ ]:
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, HumanMessage

# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", name="Lance")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Test
add_messages(initial_messages , new_message)

## Re-writing

Se enviarmos uma mensagem com o mesmo ID de uma mensagem existente na nossa lista de mensagens, ela será sobrescrita.

In [ ]:
# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model", id="1"),
                    HumanMessage(content="I'm looking for information on marine biology.", name="Lance", id="2")
                   ]

# New message to add
new_message = HumanMessage(content="I'm looking for information on whales, specifically", name="Lance", id="2")

# Test
add_messages(initial_messages , new_message)

## Remoção

Podemos remover mensagens usando RemoveMessage.

### Funcionamento do RemoveMessage
O RemoveMessage é uma classe especializada da LangChain Core que atua como uma instrução de exclusão em vez de um conteúdo de mensagem real.

Criação da Lista de Mensagens a Excluir:

~~~Python

delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]

~~~

messages[:-2] seleciona todas as mensagens, exceto as duas últimas da lista original (messages). No seu caso, ele seleciona as mensagens com id="1" e id="2".


Para cada uma dessas mensagens, um novo objeto RemoveMessage é criado, usando o id da mensagem original.

Resultado de print(delete_messages): Você verá uma lista parecida com:

~~~Python

[RemoveMessage(id='1'), RemoveMessage(id='2')]

~~~

### Aplicação da Exclusão (add_messages):

~~~Python

add_messages(messages, delete_messages)
~~~

A função add_messages (que é uma primitiva do LangChain, geralmente usada dentro do LangGraph ou de RunnableWithMessageHistory) é projetada para processar listas de mensagens que podem conter tanto objetos de mensagem padrão (HumanMessage, AIMessage) quanto objetos de exclusão (RemoveMessage).

Ao processar o delete_messages, o sistema de histórico identifica os IDs especificados nos objetos RemoveMessage ('1' e '2').

O sistema então remove as mensagens originais que correspondem a esses IDs do histórico.

Efeito Final
O histórico final (a lista messages após a operação) conterá apenas as duas últimas mensagens da lista original, pois as mensagens iniciais foram excluídas:

AIMessage("So you said you were researching ocean mammals?", name="Bot", id="3")

HumanMessage("Yes, I know about whales. But what others should I learn about?", name="Lance", id="4")

O RemoveMessage é uma maneira limpa e explícita de manipular o histórico de conversas, permitindo que você gerencie o tamanho do histórico sem precisar reconstruí-lo manualmente.

A linha de código delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]] não remove nada do histórico. Ela apenas cria uma lista de instruções sobre o que deve ser removido.

### 🗑️ Desvendando o Processo de Exclusão
1. Seleção e Instrução (Criação de delete_messages)
~~~Python

delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]

~~~

messages Original:

[0] ID: 1

[1] ID: 2

[2] ID: 3

[3] ID: 4

messages[:-2]: Esta é uma fatia que pega todos os itens exceto os dois últimos ([2] e [3]). Ele seleciona as mensagens com ID 1 e 2.

RemoveMessage(id=m.id): O list comprehension pega o ID dessas duas mensagens e cria dois objetos de instrução: RemoveMessage(id='1') e RemoveMessage(id='2').

Neste ponto, a lista original messages permanece inalterada. A lista delete_messages agora é: [RemoveMessage(id='1'), RemoveMessage(id='2')].

2. Execução da Remoção (add_messages)
~~~Python

add_messages(messages, delete_messages)
~~~

A função add_messages (ou o sistema de histórico que a executa, como no LangGraph) faz duas coisas:

Processa as Instruções: Ela recebe as instruções em delete_messages (remover IDs 1 e 2).

Aplica no Histórico: Ela aplica essas instruções no histórico principal.

Resultado: As mensagens originais com IDs 1 e 2 são removidas.

Histórico Final: O histórico resultante (que pode ser a própria lista messages ou um novo histórico dentro de um checkpointer) contém apenas as mensagens que não foram alvos de exclusão: as mensagens com IDs 3 e 4.

💡 Resposta Direta à Sua Dúvida
Você está recebendo as mensagens com ID 3 e 4 porque elas são as únicas que sobraram após você ter explicitamente criado uma lista para remover as mensagens com ID 1 e 2.

Você não removeu apenas o conteúdo, mas a estrutura inteira (o objeto da mensagem, incluindo o ID) das mensagens que você instruiu o sistema a excluir. O RemoveMessage é apenas um bilhete de instrução: "Apague o item X".

In [ ]:
from langchain_core.messages import RemoveMessage

# Message list
messages = [AIMessage("Hi.", name="Bot", id="1")]
messages.append(HumanMessage("Hi.", name="Lance", id="2"))
messages.append(AIMessage("So you said you were researching ocean mammals?", name="Bot", id="3"))
messages.append(HumanMessage("Yes, I know about whales. But what others should I learn about?", name="Lance", id="4"))

# Isolate messages to delete
delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]
print(delete_messages)